<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Лабораторная работа 5.</b> Разработка и запуск Storm приложений</div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Разработка приложений Strom с ипользованием IDE Eclipse</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Исходный код программы</a></li>
                <li><a href="#1b">Запуск проекта</a></li>
            </ol>
        </li>
        <li><a href="#2">Запуск приложения Storm</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Запуск на локальной версии</a></li>
                <li><a href="#2b">Запуск на кластере AWS</a></li>
                <li><a href="#2c">Запуск на кластере HDInsight Azure</a></li>
            </ol>
        </li>
        <li><a href="#5">Источники</a>
        </li>
    </ol>
</div>

<div>
    <p><b>Цель лабораторной работы</b> - освоить:</p>
    <ol>
        <li>создание проектов Storm с использованием среды разработки Eclipse</li>
        <li>запуск приложений Storm на кластере</li>
        <li>подключение к средствам мониторинга ресурсов Storm UI кластер</li>
    </ol>
    <p><b>Необходимые средства и схема подключения</b></p>
    <img src="img/structure_tools.png" alt="Tools for Lab 2" width="60%" style="max-width:500px;">
</div>

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Разработка приложений Storm с ипользованием IDE Eclipse</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Исходный код программы
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>WordCountTopology</p>

https://github.com/nathanmarz/storm-starter/blob/master/src/jvm/storm/starter/WordCountTopology.java

In [ ]:
package storm.starter;

import backtype.storm.Config;
import backtype.storm.LocalCluster;
import backtype.storm.StormSubmitter;
import backtype.storm.task.ShellBolt;
import backtype.storm.topology.BasicOutputCollector;
import backtype.storm.topology.IRichBolt;
import backtype.storm.topology.OutputFieldsDeclarer;
import backtype.storm.topology.TopologyBuilder;
import backtype.storm.topology.base.BaseBasicBolt;
import backtype.storm.tuple.Fields;
import backtype.storm.tuple.Tuple;
import backtype.storm.tuple.Values;
import storm.starter.spout.RandomSentenceSpout;

import java.util.HashMap;
import java.util.Map;

/**
 * This topology demonstrates Storm's stream groupings and multilang capabilities.
 */
public class WordCountTopology {
  public static class SplitSentence extends ShellBolt implements IRichBolt {

    public SplitSentence() {
      super("python", "splitsentence.py");
    }

    @Override
    public void declareOutputFields(OutputFieldsDeclarer declarer) {
      declarer.declare(new Fields("word"));
    }

    @Override
    public Map<String, Object> getComponentConfiguration() {
      return null;
    }
  }

  public static class WordCount extends BaseBasicBolt {
    Map<String, Integer> counts = new HashMap<String, Integer>();

    @Override
    public void execute(Tuple tuple, BasicOutputCollector collector) {
      String word = tuple.getString(0);
      Integer count = counts.get(word);
      if (count == null)
        count = 0;
      count++;
      counts.put(word, count);
      collector.emit(new Values(word, count));
    }

    @Override
    public void declareOutputFields(OutputFieldsDeclarer declarer) {
      declarer.declare(new Fields("word", "count"));
    }
  }

  public static void main(String[] args) throws Exception {

    TopologyBuilder builder = new TopologyBuilder();

    builder.setSpout("spout", new RandomSentenceSpout(), 5);

    builder.setBolt("split", new SplitSentence(), 8).shuffleGrouping("spout");
    builder.setBolt("count", new WordCount(), 12).fieldsGrouping("split", new Fields("word"));

    Config conf = new Config();
    conf.setDebug(true);


    if (args != null && args.length > 0) {
      conf.setNumWorkers(3);

      StormSubmitter.submitTopology(args[0], conf, builder.createTopology());
    }
    else {
      conf.setMaxTaskParallelism(3);

      LocalCluster cluster = new LocalCluster();
      cluster.submitTopology("word-count", conf, builder.createTopology());

      Thread.sleep(10000);

      cluster.shutdown();
    }
  }
}

<p>RandomSentenceSpout</p>

https://github.com/nathanmarz/storm-starter/blob/master/src/jvm/storm/starter/spout/RandomSentenceSpout.java

In [ ]:
package storm.starter.spout;

import backtype.storm.spout.SpoutOutputCollector;
import backtype.storm.task.TopologyContext;
import backtype.storm.topology.OutputFieldsDeclarer;
import backtype.storm.topology.base.BaseRichSpout;
import backtype.storm.tuple.Fields;
import backtype.storm.tuple.Values;
import backtype.storm.utils.Utils;

import java.util.Map;
import java.util.Random;

public class RandomSentenceSpout extends BaseRichSpout {
  SpoutOutputCollector _collector;
  Random _rand;


  @Override
  public void open(Map conf, TopologyContext context, SpoutOutputCollector collector) {
    _collector = collector;
    _rand = new Random();
  }

  @Override
  public void nextTuple() {
    Utils.sleep(100);
    String[] sentences = new String[]{ "the cow jumped over the moon", "an apple a day keeps the doctor away",
        "four score and seven years ago", "snow white and the seven dwarfs", "i am at two with nature" };
    String sentence = sentences[_rand.nextInt(sentences.length)];
    _collector.emit(new Values(sentence));
  }

  @Override
  public void ack(Object id) {
  }

  @Override
  public void fail(Object id) {
  }

  @Override
  public void declareOutputFields(OutputFieldsDeclarer declarer) {
    declarer.declare(new Fields("word"));
  }

}

<a name = "1b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Запуск проекта
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Самостоятельное добавлние пакетов Hadoop в проект</p>

Java -> JAR <br>
https://www.cs.utexas.edu/~scottm/cs307/handouts/Eclipse%20Help/jarInEclipse.htm

Сборка зависимостей с использованием Maven

http://tutorials.pcdhan.com/hadoop/developing-your-first-mapreduce-app-in-java-using-eclipse-and-maven/

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Запуск приложения Storm</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "2a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Запуск на локальной версии
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2b">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
    <p><b>Схема подключения</b></p>
    <img src="img/connection_cloudera.png" alt="Tools for Lab 5" width="30%" style="max-width:300px;min-width:200px;">
</div>

<p><b>Задание</b></p>

<ol>
    <li>Запуск локальной версии Storm</li>
    <li>Запуск Storm приложения</li>
    <li>Подключение к панели UI Storm для мониторинга выполнения</li>
</ol>

<a name = "2b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Запуск на кластере EMR AWS  
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2c">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
    <p><b>Схема подключения</b></p>
    <img src="img/connection_aws.png" alt="Tools for Lab 5" width="50%" style="max-width:500px;min-width:300px;">
</div>
</div>

<p><b>Задание</b></p>

<ol>
    <li>Развертывание кластера EMR</li>
    <li>Разверетывание Storm на AWS
        <ul>
            <li>http://storm.apache.org/about/deployment.html</li>
            <li>https://github.com/nathanmarz/storm-deploy</li>
        </ul>
    </li>
    <li>Запуск Storm приложения</li>
    <li>Подключение к панели UI Storm для мониторинга выполнения</li>
</ol>

<a name = "2с"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            с. Запуск на кластере HDInsight Azure  
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2b">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
    <p><b>Схема подключения</b></p>
    <img src="img/connection_aws.png" alt="Tools for Lab 5" width="50%" style="max-width:500px;min-width:300px;">
</div>
</div>

<p><b>Задание</b></p>

<ol>
    <li>Развертывание кластера Storm на HDInsight Azure</li>
    <li>Запуск Storm приложения</li>
    <li>Подключение к панели UI Storm для мониторинга выполнения</li>
</ol>

<a href="http://storm.apache.org/releases/1.0.0/Running-topologies-on-a-production-cluster.html">Running Topologies on a Production Cluster</a><br>

<a href="https://azure.microsoft.com/ru-ru/documentation/articles/hdinsight-apache-storm-tutorial-get-started/">Учебник по Apache Storm в HDInsight: начало работы с анализом больших объемов данных в HDInsight с помощью примеров Storm Starter</a><br>

<a href="https://azure.microsoft.com/en-us/documentation/articles/hdinsight-storm-deploy-monitor-topology-linux/">Deploy and manage Apache Storm topologies on Linux-based HDInsight</a><br>

<a href="https://azure.microsoft.com/ru-ru/documentation/articles/hdinsight-storm-develop-java-topology/">Разработка топологий на основе Java для базовых приложений подсчета слов с помощью Apache Storm и Maven в HDInsight</a><br>

<a href="https://azure.microsoft.com/ru-ru/documentation/articles/hdinsight-storm-twitter-trending/">Определение популярных тем в Twitter с помощью Apache Storm в HDInsight</a>